# Estrazione delle quadruple (aspect-opinion-category-sentiment) con ModernBERT su Restaurant-ACOS

In [1]:
# Import delle librerie necessarie
import torch
import numpy as np
import random
import pandas as pd
import wandb
import os
import sys
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, get_linear_schedule_with_warmup, AutoModel
from torch.optim import AdamW
import torch.nn as nn
import bitsandbytes as bnb
import pickle
from evaluate import load
from tqdm import tqdm
from torch.amp import autocast, GradScaler # Per Mixed Precision
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
from torchcrf import CRF  # Ricordati di fare !pip install pytorch-crf

# --- 1. CONFIGURAZIONE DEL DEVICE ---
# Se hai una GPU NVIDIA, userà 'cuda'. Se hai un Mac M1/M2, userà 'mps'. Altrimenti 'cpu'.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f" GPU Trovata: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(" Acceleratore Apple Metal (MPS) Trovato")
else:
    device = torch.device("cpu")
    print(" Nessuna GPU trovata. L'addestramento sarà lento.")

print("Librerie caricate.")

 Acceleratore Apple Metal (MPS) Trovato
Librerie caricate.


### Impostazioni per la riproducibilità 

In [2]:
def set_seed(seed_value=42):
    """Imposto i seed per la riproducibilità."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        # Imposto anche i seed per la GPU, se disponibile
        torch.cuda.manual_seed_all(seed_value)

# Esegui l'impostazione del seed
set_seed(42) 
print("Random seeds impostati su 42.")

Random seeds impostati su 42.


In [8]:
if wandb.run is not None:
    wandb.finish()

WANDB_ENTITY = "cristinatextmining"

# 1. Definizione degli Hyperparameters
config = {
    "learning_rate": 2e-5,
    "epochs": 40,
    "batch_size": 16,
    "model_name": "answerdotai/ModernBERT-base",
    "dataset": "Restaurant-ACOS",  
    "seed": 42,
    'patience': 5  # Per Early Stopping
}

# 2. Inizializzazione del Run
wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    config=config,
    name=f"Step1_Class_{config['model_name']}_{config['dataset']}" 
)

print(f"W&B inizializzato per il progetto: {wandb.run.project}")
print(f"Nome della Run attuale: {wandb.run.name}")

wandb: Currently logged in as: cristinatomaciello2001 (cristinatextmining) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B inizializzato per il progetto: BigData-TextMining-ACOS
Nome della Run attuale: Step1_Class_answerdotai/ModernBERT-base_Restaurant-ACOS


## PyTorch Dataset & DataLoader Construction

### Creazione di PyTorch Dataset e DataLoader
Questa cella si occupa di caricare i dati pre-processati e di "impacchettarli" nel formato esatto richiesto dalla nostra nuova architettura PyTorch personalizzata. Rappresenta un passaggio cruciale per replicare fedelmente il paper originale, permettendoci di gestire gli elementi impliciti.

Nello specifico, il codice esegue tre operazioni fondamentali:

1. **Caricamento dei DataFrame:** Legge i file `.pkl` (Train, Dev e Test per il dominio Laptop) che abbiamo precedentemente aggiornato. Questi file ora contengono le annotazioni binarie che indicano la presenza di aspetti o opinioni implicite.
2. **Definizione della Classe Custom `ACOSDataset`:** Questa è la modifica principale rispetto a una pipeline standard di HuggingFace. Invece di restituire solo i classici 3 tensori della Token Classification (`input_ids`, `attention_mask` e `labels` con i tag BIO), questa classe sovrascritta restituisce **5 tensori** per ogni frase. Vengono infatti estratti e passati al modello anche `implicit_aspect_labels` e `implicit_opinion_labels`. Questi tensori serviranno ad addestrare in parallelo le due teste di classificazione binaria sul token `[CLS]`.
3. **Configurazione dei DataLoader:** Crea gli iteratori di PyTorch che alimenteranno il modello durante l'addestramento e il test, processando blocchi (batch) di 16 frasi alla volta. I dati di training vengono rimescolati (`shuffle=True`) per stabilizzare l'apprendimento della rete.

In [3]:
# --- 1. CARICAMENTO DEI DATASET SALVATI  ---
cartella_dati = "data_allineati"

print("Caricamento dei dataset pre-processati...")
# Carichiamo i Ristoranti
df_train_align_rest = pd.read_pickle(os.path.join(cartella_dati, "train_rest_aligned.pkl"))
df_dev_align_rest = pd.read_pickle(os.path.join(cartella_dati, "dev_rest_aligned.pkl"))
df_test_align_rest = pd.read_pickle(os.path.join(cartella_dati, "test_rest_aligned.pkl"))


class ACOSDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['labels'].tolist()
        # Estraiamo le colonne per gli impliciti!
        self.implicit_aspect_label = df['implicit_aspect_label'].tolist()
        self.implicit_opinion_label = df['implicit_opinion_label'].tolist()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            # Passiamo le etichette al Dataloader
            'implicit_aspect_labels': torch.tensor(self.implicit_aspect_label[idx], dtype=torch.long),
            'implicit_opinion_labels': torch.tensor(self.implicit_opinion_label[idx], dtype=torch.long)
        }

# --- CREAZIONE DELLE ISTANZE ---

# Creiamo i dataset per il dominio restaruant
train_dataset_rest = ACOSDataset(df_train_align_rest)
dev_dataset_rest = ACOSDataset(df_dev_align_rest)
test_dataset_rest = ACOSDataset(df_test_align_rest)

# --- CONFIGURAZIONE DATALOADERS ---

BATCH_SIZE = 16 # Numero di frasi analizzate contemporaneamente

train_loader_rest = DataLoader(train_dataset_rest, batch_size=BATCH_SIZE, shuffle=True)
dev_loader_rest = DataLoader(dev_dataset_rest, batch_size=BATCH_SIZE)
test_loader_rest = DataLoader(test_dataset_rest, batch_size=BATCH_SIZE)

print(f"Dataset e DataLoaders creati con successo!")
print(f"Esempi nel set di Training RESTAURANT: {len(train_dataset_rest)}")  

Caricamento dei dataset pre-processati...
Dataset e DataLoaders creati con successo!
Esempi nel set di Training RESTAURANT: 1530


### Architettura Multi-Task: ModernBERT ACOS Extractor

In questa cella definiamo il cuore dello Step 1 (L'Investigatore) della nostra pipeline. Invece di usare un modello standard pre-confezionato, abbiamo costruito un'architettura di rete neurale custom basata su ModernBERT per risolvere tre task contemporaneamente (Multi-Task Learning), eguagliando l'approccio dei paper State-of-the-Art.

Ecco le innovazioni matematiche e strutturali introdotte in questa classe:

* Layer CRF (Conditional Random Field): Il vero "game changer". Invece di usare una semplice CrossEntropyLoss che classifica ogni parola in isolamento, abbiamo inserito un layer statistico che impara le "regole" delle etichette BIO (es. è impossibile che un tag I-ASP appaia se prima non c'è stato un B-ASP). Agisce come un potentissimo "correttore ortografico" per l'estrazione degli span, massimizzando il Recall.

* Teste per gli Impliciti sul [CLS]: Abbiamo agganciato due classificatori binari indipendenti al token speciale [CLS] (che racchiude il significato dell'intera frase) per determinare se l'aspetto o l'opinione sono sottointesi (-1, -1).

* Loss Combinata e Bilanciata: Durante l'addestramento, il modello calcola la Negative Log-Likelihood del CRF e la somma alla Loss dei due task impliciti. Per risolvere il forte sbilanciamento del dataset, abbiamo assegnato un peso 5.0 alla classe "Implicito" (implicit_weights = [1.0, 5.0]), forzando il modello a non ignorare questi casi rari.

* Decodifica di Viterbi: In fase di inferenza (test), la rete non emette probabilità grezze da filtrare con argmax, ma usa l'algoritmo di Viterbi per calcolare matematicamente la sequenza grammaticale perfetta, restituendola già decodificata e pronta all'uso.

* Memory Optimization: Instanziamo l'ottimizzatore AdamW8bit della libreria bitsandbytes per permettere l'addestramento di questa complessa struttura Multi-Task senza saturare la memoria VRAM (Out Of Memory).

In [4]:
# Definiamo le 5 etichette: 0=O, 1=B-ASP, 2=I-ASP, 3=B-OPI, 4=I-OPI
NUM_LABELS = 5 

class ModernBertACOS_Extractor(nn.Module):
    def __init__(self, model_name="answerdotai/ModernBERT-base", num_labels=5):
        super().__init__()
        # Carichiamo la "schiena" del modello (l'encoder base)
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size
        
        # Testolina 1: Emette i "punteggi grezzi" per il CRF
        self.token_classifier = nn.Linear(hidden_size, num_labels)
        
        # IL LAYER CRF (Correttore Ortografico per Sequenze) 
        self.crf = CRF(num_labels, batch_first=True)
        
        # Testoline 2 e 3: Indovinano se ci sono impliciti (Manteniamo la tua intuizione!)
        self.implicit_aspect_classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(hidden_size, 2)
        )
        self.implicit_opinion_classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(hidden_size, 2)
        )
        
    def forward(self, input_ids, attention_mask, labels=None, implicit_aspect_labels=None, implicit_opinion_labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state 
        
        # Prendiamo il token [CLS] (posizione 0) per i classificatori binari
        cls_output = sequence_output[:, 0, :] 
        
        # Punteggi grezzi (emissions) per il CRF
        emissions = self.token_classifier(sequence_output)
        
        # Le due testoline calcolano i logit per gli impliciti
        imp_asp_logits = self.implicit_aspect_classifier(cls_output)
        imp_opi_logits = self.implicit_opinion_classifier(cls_output)
        
        loss = None
        
        # --- FASE DI ADDESTRAMENTO (Calcolo della Loss) ---
        if labels is not None and implicit_aspect_labels is not None and implicit_opinion_labels is not None:
            device = input_ids.device
            
            # --- NOVITÀ 1: LOSS DEL CRF ---
            # Il CRF non tollera le label "-100" (usate spesso per il padding in HF).
            # Creiamo una maschera valida e sostituiamo i -100 con 0 per sicurezza.
            valid_mask = (labels >= 0) & attention_mask.bool()
            safe_labels = torch.where(labels >= 0, labels, torch.zeros_like(labels))
            
            # Calcoliamo la loss negativa della log-likelihood del CRF
            loss_token = -self.crf(emissions, safe_labels, mask=valid_mask, reduction='mean')
            
            # --- NOVITÀ 2: Pesi per gli Impliciti (Intatti!) ---
            # Classe 0 (Esplicito) = 1.0. Classe 1 (Implicito) = 5.0.
            implicit_weights = torch.tensor([1.0, 5.0], device=device)
            loss_fct_implicit = nn.CrossEntropyLoss(weight=implicit_weights)
            
            loss_asp = loss_fct_implicit(imp_asp_logits, implicit_aspect_labels)
            loss_opi = loss_fct_implicit(imp_opi_logits, implicit_opinion_labels)
            
            # --- NOVITÀ 3: Moltiplicatori della Loss Multi-Task ---
            loss = loss_token + (1.5 * loss_asp) + (2.0 * loss_opi)
            
            
        # --- FASE DI INFERENZA (Decodifica) ---
        # Usiamo l'algoritmo di Viterbi per trovare la sequenza grammaticalmente perfetta!
        mask_crf = attention_mask.bool()
        token_preds = self.crf.decode(emissions, mask=mask_crf)
        
        # Poiché il CRF restituisce liste di lunghezza variabile (taglia via il padding),
        # le ri-paddiamo con zeri per restituire un tensore uniforme e non rompere il tuo test.
        batch_size = input_ids.shape[0]
        max_seq_length = input_ids.shape[1]
        padded_preds = []
        
        for i in range(batch_size):
            pred = token_preds[i]
            pad_len = max_seq_length - len(pred)
            padded_preds.append(pred + [0] * pad_len)
            
        # Restituiamo il tensore finale
        token_preds_tensor = torch.tensor(padded_preds, device=input_ids.device)
            
        return {
            "loss": loss, 
            "token_logits": token_preds_tensor,  # ORA CONTIENE GIA' I TAG DECISI!
            "imp_asp_logits": imp_asp_logits, 
            "imp_opi_logits": imp_opi_logits
        }

print("Scaricamento e configurazione della nuova architettura Multi-Task ModernBERT...")

# Inizializziamo il nostro modello custom invece di AutoModelForTokenClassification
model_step1 = ModernBertACOS_Extractor(num_labels=NUM_LABELS)

# Spostiamo il modello sul dispositivo di calcolo (GPU/MPS/CPU)
model_step1.to(device)

# --- 2. CONFIGURAZIONE DELL'OTTIMIZZATORE E DELLA LOSS ---

# Manteniamo la tua ottima scelta di usare l'optimizer a 8-bit per non saturare la memoria!
optimizer = bnb.optim.AdamW8bit(model_step1.parameters(), lr=2e-5)

print("\n" + "="*50)
print("MODELLO MULTI-TASK (CON CRF) PRONTO PER IL TRAINING")
print("="*50)
print(f"Architettura: ModernBERT-base + Layer CRF (Custom ACOS Extractor)")
print(f"Task: CRF Sequence Labeling + 2x Binary Classification (Impliciti)")
print(f"Numero di Classi Token: {NUM_LABELS}")
print(f"Optimizer: AdamW 8-bit (lr=2e-5)")
print(f"Loss Function: Neg Log-Likelihood (CRF) + 2x CrossEntropy (Impliciti)")
print(f"Device: {device}")
print("="*50)


Scaricamento e configurazione della nuova architettura Multi-Task ModernBERT...


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
decoder.bias      | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
head.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



MODELLO MULTI-TASK (CON CRF) PRONTO PER IL TRAINING
Architettura: ModernBERT-base + Layer CRF (Custom ACOS Extractor)
Task: CRF Sequence Labeling + 2x Binary Classification (Impliciti)
Numero di Classi Token: 5
Optimizer: AdamW 8-bit (lr=2e-5)
Loss Function: Neg Log-Likelihood (CRF) + 2x CrossEntropy (Impliciti)
Device: mps


In [ ]:
# --- 1. CONFIGURAZIONE AVANZATA MEMORIA E WANDB ---
print("Attivazione Ottimizzazioni di Memoria...")

# Gradient Checkpointing: si applica all'encoder interno (BERT) della nostra classe custom
model_step1.bert.gradient_checkpointing_enable()

accumulation_steps = wandb.config.get('accumulation_steps', 4) 
patience = wandb.config.get('patience', 5)
epochs = wandb.config.get('epochs', 40)
lr = wandb.config.get('learning_rate', 2e-5)
patience_counter = 0

# Optimizer a 8-bit
optimizer = bnb.optim.AdamW8bit(model_step1.parameters(), lr=lr)

# Scaler per Mixed Precision (FP16)
scaler = GradScaler() 

total_steps = (len(train_loader_rest) // accumulation_steps) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps
)

# --- 2. FUNZIONI DI SUPPORTO OTTIMIZZATE (MULTI-TASK) ---

def evaluate_model_multitask(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            imp_asp_labels = batch['implicit_aspect_labels'].to(device)
            imp_opi_labels = batch['implicit_opinion_labels'].to(device)
            
            # Usiamo autocast in valutazione per risparmiare memoria
            #nel caso il trianing dia errori di memoria, commenta la riga con autocast
            with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(
                    input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    labels=labels,
                    implicit_aspect_labels=imp_asp_labels,
                    implicit_opinion_labels=imp_opi_labels
                )
            
            total_loss += outputs['loss'].item()
    return total_loss / len(data_loader)

def train_epoch_multitask(model, data_loader, optimizer, scheduler, device, epoch_idx, scaler, accumulation_steps):
    model.train()
    total_loss = 0
    optimizer.zero_grad() # Reset iniziale
    
    loop = tqdm(data_loader, leave=True)
    
    for i, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        imp_asp_labels = batch['implicit_aspect_labels'].to(device)
        imp_opi_labels = batch['implicit_opinion_labels'].to(device)
        
        # A. Mixed Precision Forward Pass (FP16)
        #nel caso il trianing dia errori di memoria, commenta la riga con autocast
        with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            outputs = model(
                input_ids=input_ids, 
                attention_mask=attention_mask, 
                labels=labels,
                implicit_aspect_labels=imp_asp_labels,
                implicit_opinion_labels=imp_opi_labels
            )
            # Normalizziamo la loss per l'accumulo dei gradienti
            loss = outputs['loss'] / accumulation_steps 
        
        # B. Backward Pass con Scaler (Evita underflow/overflow dell'FP16)
        scaler.scale(loss).backward()
        
        # C. Update Pesi ogni 'accumulation_steps'
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(data_loader):
            
            # GRADIENT CLIPPING
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            #nel caso di errori di memoria, commenta le righe con scaler.step
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item() * accumulation_steps
        wandb.log({"batch_loss": loss.item() * accumulation_steps})
        loop.set_description(f"Epoca {epoch_idx + 1}")
        loop.set_postfix(loss=loss.item() * accumulation_steps)

    return total_loss / len(data_loader)

# --- 3. CICLO DI ADDESTRAMENTO ---

print(f"\nTraining Multi-Task su RESTAURANT: {epochs} epoche | Device: {device}")
print(f"Accumulo Gradienti: ogni {accumulation_steps} step | FP16: Attivato")

best_valid_loss_rest = float('inf')
output_dir = "./best_multitask_extractor_restaurant"

for epoch in range(epochs):
    print(f"\n--- Epoca {epoch+1}/{epochs} ---")
    
    # 1. Training
    train_loss_rest = train_epoch_multitask(model_step1, train_loader_rest, optimizer, scheduler, device, epoch, scaler, accumulation_steps)
    
    # 2. Validazione
    valid_loss_rest = evaluate_model_multitask(model_step1, dev_loader_rest, device)
    
    # Pulizia spietata della cache della GPU a fine epoca!
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print(f"Train Loss: {train_loss_rest:.4f} | Valid Loss: {valid_loss_rest:.4f}")
    
    # 3. Log metriche epoca su W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss_epoch": train_loss_rest,
        "valid_loss_epoch": valid_loss_rest
    })
    
    # --- LOGICA EARLY STOPPING & CHECKPOINT ---
    
    if valid_loss_rest < best_valid_loss_rest:
        best_valid_loss_rest = valid_loss_rest
        patience_counter = 0  
        
        print(f"Miglior modello trovato (Loss: {best_valid_loss_rest:.4f})")
        
        os.makedirs(output_dir, exist_ok=True)
        # Salvataggio Custom per l'architettura Multi-Task
        torch.save(model_step1.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
        
    else:
        patience_counter += 1  
        print(f"Nessun miglioramento. Patience: {patience_counter}/{patience}")
        
        if patience_counter >= patience:
            print(f"\nEARLY STOPPING ATTIVATO! Interruzione all'epoca {epoch+1}.")
            break 

print("\nFine Addestramento Multi-Task.")
wandb.finish()

Attivazione Ottimizzazioni di Memoria...

Training Multi-Task su RESTAURANT: 40 epoche | Device: cuda
Accumulo Gradienti: ogni 4 step | FP16: Attivato

--- Epoca 1/40 ---


Epoca 1:   3%|▎         | 3/96 [00:01<00:39,  2.36it/s, loss=40.6]/home/al3th3ia/Scrivania/Cristina/progetto-text-mining/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Epoca 1: 100%|██████████| 96/96 [00:30<00:00,  3.18it/s, loss=23.3]


Train Loss: 33.0723 | Valid Loss: 17.7910
Miglior modello trovato (Loss: 17.7910)

--- Epoca 2/40 ---


Epoca 2: 100%|██████████| 96/96 [00:29<00:00,  3.27it/s, loss=17.3]


Train Loss: 17.0509 | Valid Loss: 12.5556
Miglior modello trovato (Loss: 12.5556)

--- Epoca 3/40 ---


...

Epoca 10: 100%|██████████| 96/96 [00:29<00:00,  3.21it/s, loss=0.919]


Train Loss: 1.1301 | Valid Loss: 8.6259
Nessun miglioramento. Patience: 4/5

--- Epoca 11/40 ---


Epoca 11: 100%|██████████| 96/96 [00:29<00:00,  3.23it/s, loss=0.728]


Train Loss: 0.6777 | Valid Loss: 9.0897
Nessun miglioramento. Patience: 5/5

EARLY STOPPING ATTIVATO! Interruzione all'epoca 11.

Fine Addestramento Multi-Task.


batch_loss,█▇▆▆▅▅▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▂▂▃▄▅▅▆▇▇█
train_loss_epoch,█▅▄▃▂▂▂▁▁▁▁
valid_loss_epoch,█▅▃▂▁▁▁▂▂▂▂
batch_loss,0.72757
epoch,11
train_loss_epoch,0.67766
valid_loss_epoch,9.08971


In [12]:
# --- A. CARICAMENTO DEL "CAMPIONE" MULTI-TASK ---
print("Caricamento del modello Multi-Task migliore...")

# 1. Inizializziamo la nostra architettura custom
model_step1 = ModernBertACOS_Extractor(num_labels=5)

# 2. Carichiamo i pesi salvati del miglior modello (solo i pesi, non l'intero oggetto)
model_path = "./best_multitask_extractor_restaurant/pytorch_model.bin"
model_step1.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))

model_step1.to(device)
model_step1.eval() # Modalità esame (spegne dropout)

# --- B. PREPARAZIONE METRICHE ---
# Carichiamo la metrica seqeval (standard per NER/ABSA)
metric = load("seqeval")

# Mappa per decodificare i numeri in etichette
id2label = {0: 'O', 1: 'B-ASP', 2: 'I-ASP', 3: 'B-OPI', 4: 'I-OPI'}

print("\nInizio Test Multi-Task sul Dataset Restaurant...")

# --- C. CICLO DI PREVISIONE ---
predictions_tokens = []
true_labels_tokens = []

# Liste per salvare le predizioni binarie (Impliciti)
true_imp_asp, pred_imp_asp = [], []
true_imp_opi, pred_imp_opi = [], []

with torch.no_grad():
    for batch in tqdm(test_loader_rest, desc="Test RESTAURANT"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        imp_asp_labels = batch['implicit_aspect_labels'].to(device)
        imp_opi_labels = batch['implicit_opinion_labels'].to(device)

        # 1. Il modello fa le sue 3 predizioni contemporaneamente
        with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            outputs = model_step1(input_ids, attention_mask=attention_mask)
        
        # 2. Estraiamo i risultati dalle 3 testoline
        token_logits = outputs['token_logits']
        token_preds = token_logits # Il CRF ha già scelto!
        asp_preds = torch.argmax(outputs['imp_asp_logits'], dim=-1)
        opi_preds = torch.argmax(outputs['imp_opi_logits'], dim=-1)
        
        # Salviamo i risultati binari
        true_imp_asp.extend(imp_asp_labels.cpu().tolist())
        pred_imp_asp.extend(asp_preds.cpu().tolist())
        
        true_imp_opi.extend(imp_opi_labels.cpu().tolist())
        pred_imp_opi.extend(opi_preds.cpu().tolist())

        # 3. Convertiamo i numeri in etichette BIO (pulendo il padding)
        for i in range(len(labels)):
            true_label_row = []
            pred_label_row = []
            
            for j in range(len(labels[i])):
                # Ignoriamo i token di padding (dove attention_mask è 0 o la label è -100)
                if labels[i][j] != -100 and attention_mask[i][j] == 1: 
                    true_label_row.append(id2label[labels[i][j].item()])
                    pred_label_row.append(id2label[token_preds[i][j].item()])
            
            true_labels_tokens.append(true_label_row)
            predictions_tokens.append(pred_label_row)

# --- D. CALCOLO E STAMPA RISULTATI ---
results_seq = metric.compute(predictions=predictions_tokens, references=true_labels_tokens)

print("\n" + "="*60)
print("RISULTATI FINALI: TOKEN CLASSIFICATION (Parole Esplicite)")
print("="*60)

print(f"Overall Precision: {results_seq['overall_precision']:.4f}")
print(f"Overall Recall:    {results_seq['overall_recall']:.4f}")
print(f"Overall F1-Score:  {results_seq['overall_f1']:.4f}")

print("\nDettaglio per Classe (Quello che conta per il paper):")
print("-" * 50)
for key in results_seq.keys():
    if key in ['ASP', 'OPI']: 
        print(f"   {key}:")
        print(f"   Precision: {results_seq[key]['precision']:.4f}")
        print(f"   Recall:    {results_seq[key]['recall']:.4f}")
        print(f"   F1-Score:  {results_seq[key]['f1']:.4f}")
        print(f"   Support:   {results_seq[key]['number']}") 

print("\n" + "="*60)
print("RISULTATI FINALI: IDENTIFICAZIONE IMPLICITI (NULL)")
print("="*60)

# Metriche per Aspetti Impliciti
acc_asp = accuracy_score(true_imp_asp, pred_imp_asp)
print(f"Accuratezza Aspetti Impliciti: {acc_asp:.4f}")
print(classification_report(true_imp_asp, pred_imp_asp, target_names=["Esplicito (0)", "Implicito (1)"], zero_division=0))

print("-" * 50)

# Metriche per Opinioni Implicite
acc_opi = accuracy_score(true_imp_opi, pred_imp_opi)
print(f"Accuratezza Opinioni Implicite: {acc_opi:.4f}")
print(classification_report(true_imp_opi, pred_imp_opi, target_names=["Esplicito (0)", "Implicito (1)"], zero_division=0))
print("="*60)

Caricamento del modello Multi-Task migliore...


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
decoder.bias      | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
head.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Inizio Test Multi-Task sul Dataset Restaurant...


Test RESTAURANT: 100%|██████████| 37/37 [00:06<00:00,  6.13it/s]


RISULTATI FINALI: TOKEN CLASSIFICATION (Parole Esplicite)
Overall Precision: 0.6066
Overall Recall:    0.6616
Overall F1-Score:  0.6329

Dettaglio per Classe (Quello che conta per il paper):
--------------------------------------------------
   ASP:
   Precision: 0.5634
   Recall:    0.5921
   F1-Score:  0.5774
   Support:   608
   OPI:
   Precision: 0.6443
   Recall:    0.7269
   F1-Score:  0.6831
   Support:   648

RISULTATI FINALI: IDENTIFICAZIONE IMPLICITI (NULL)
Accuratezza Aspetti Impliciti: 0.5369
               precision    recall  f1-score   support

Esplicito (0)       0.95      0.32      0.48       386
Implicito (1)       0.42      0.96      0.58       197

     accuracy                           0.54       583
    macro avg       0.68      0.64      0.53       583
 weighted avg       0.77      0.54      0.51       583

--------------------------------------------------
Accuratezza Opinioni Implicite: 0.5352
               precision    recall  f1-score   support

Esplicito 

## Classificatore Category-Sentiment (Extract-Classify-ACOS)

Implementiamo il **secondo stadio** dell'architettura proposta nel paper. Dopo aver estratto gli Aspetti e le Opinioni nello Step 1, ora dobbiamo capire a quale Categoria appartengono e qual è il loro Sentiment.

Il codice di preparazione è diviso in tre componenti fondamentali:

 1. Il Dataset PyTorch (`ACOSPairDataset`)

 2. L'Architettura Custom (`ModernBertACOSClassifier`)

 3. Inizializzazione e DataLoaders

### Creazione del Dataset e DataLoader per il training e il testing
In questa cella definiamo la classe ACOSPairDataset per alimentare lo Step 2. Poiché abbiamo deciso di utilizzare un'architettura Cross-Encoder, il modo in cui prepariamo i dati è completamente diverso e molto più elegante rispetto agli approcci classici.

Invece di passare al modello gli indici numerici di dove si trovano l'aspetto e l'opinione nella frase, traduciamo le coordinate in vero e proprio testo, creando una "seconda frase" artificiale da affiancare alla recensione originale.

Ecco i passaggi chiave ("La Magia Cross-Encoder"):

* Estrazione delle Parole: Il codice prende gli indici numerici (a_span, o_span) e li usa per ritagliare le parole esatte dalla frase originale. Se un elemento è implicito (coordinate -1), viene convertito automaticamente nella stringa testuale "null".

* Creazione del Prompt (Cross-Text): Viene creata una nuova stringa di contesto formattata esattamente così: aspect: [parola] opinion: [parola].

* Tokenizzazione a Doppia Frase: Sfruttiamo una funzionalità nativa (e potentissima) del Tokenizer di HuggingFace. Passandogli due stringhe separate (text e cross_text), il tokenizer le unisce in automatico inserendo il token separatore in mezzo:
[CLS] Testo della recensione originale [SEP] aspect: pizza opinion: buonissima [SEP]

* Semplificazione dell'Output: Grazie a questa formattazione testuale, il modello leggerà contemporaneamente la frase e la coppia da analizzare, calcolandone l'interazione tramite la Self-Attention. Di conseguenza, il Dataset restituisce solo gli input_ids e le labels, senza più bisogno di complicati tensori con le posizioni degli span!

In [5]:
class ACOSPairDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['review_text']
        
        # Estraiamo gli span (togliendo il +1 che avevamo messo per il [CLS] 
        # perché ora ci servono per tagliare la stringa originale)
        a_span = row['aspect_span']
        o_span = row['opinion_span']
        
        words = text.split()
        a_start, a_end = a_span[0] - 1, a_span[1] - 1
        o_start, o_end = o_span[0] - 1, o_span[1] - 1
        
        # Estraiamo le parole (o "null" se è implicito)
        aspect_str = " ".join(words[a_start:a_end]) if a_start >= 0 else "null"
        opinion_str = " ".join(words[o_start:o_end]) if o_start >= 0 else "null"
        
        # MAGIA CROSS-ENCODER: Creiamo la stringa contesto!
        cross_text = f"aspect: {aspect_str} opinion: {opinion_str}"

        # Il tokenizer unirà il 'text' e il 'cross_text' in automatico
        encoding = self.tokenizer(
            text,
            cross_text, # Passiamo la seconda stringa!
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        labels = torch.tensor(row['labels'], dtype=torch.long)

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
            # Niente più aspect_span e opinion_span da passare!
        }

### 2. L'Architettura Custom (`ModernBertACOSClassifier`)
In questa cella definiamo il modello per lo Step 2 della nostra pipeline. Il suo compito è prendere in input la frase originale abbinata a una specifica coppia Aspetto/Opinione e determinare a quale Categoria appartiene e con quale Sentimento.

Rispetto al codice originale del paper del 2021, questa architettura presenta un upgrade ingegneristico fondamentale: Il passaggio da Bi-Encoder a Cross-Encoder.

Ecco le innovazioni chiave di questa classe:

* Architettura Cross-Encoder (Dimensione 768): Nel paper originale, gli autori estraevano i vettori separati dell'aspetto e dell'opinione (768 + 768) e li concatenavano in un vettore da 1536 dimensioni (Bi-Encoder). Noi, invece, passiamo al modello un'unica stringa formattata (es. Testo recensione + "aspect: X opinion: Y"). Questo permette al meccanismo di Self-Attention di calcolare l'interazione tra la coppia e il testo in modo nativo. Di conseguenza, ci basta usare il singolo vettore [CLS] da 768 dimensioni!

* Weight Sharing (Continuità di Apprendimento): Invece di inizializzare un ModernBERT da zero, carichiamo i pesi del modello "vincitore" dallo Step 1 (path_to_best_model). In questo modo, lo Step 2 eredita tutta la comprensione linguistica e sintattica che l'Investigatore ha già imparato sul nostro dominio specifico.

* Teste di Classificazione Multiple (ModuleList): Abbiamo creato dinamicamente un array di teste lineari indipendenti, una per ogni singola categoria del dataset (es. 121 teste per i ristoranti).

* Filtro Integrato (Classi di output = 4): Ogni testa restituisce 4 logit. Le prime tre classi rappresentano il Sentimento (Positivo, Negativo, Neutro). La quarta classe (Classe 3) funge da "Buttafuori": se il modello ritiene che la coppia Aspetto/Opinione sia falsa o non c'entri nulla con quella categoria, la classifica come "Invalida", scartando i falsi positivi generati dal prodotto cartesiano dello Step 1.

In [6]:
class ModernBertACOSClassifier(nn.Module):
    def __init__(self, path_to_best_model, num_categories):
        super(ModernBertACOSClassifier, self).__init__()
        
        # Carichiamo il corpo dal modello Step 1
        self.modernbert = AutoModel.from_pretrained(path_to_best_model)
        hidden_size = self.modernbert.config.hidden_size # 768
        
        # Le 121 teste ORA PRENDONO 768 (non più 1536)
        self.heads = nn.ModuleList([
            nn.Linear(hidden_size, 4) for _ in range(num_categories)
        ])
        
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask): # Span rimossi dai parametri!
        outputs = self.modernbert(input_ids=input_ids, attention_mask=attention_mask)
        
        # Prendiamo semplicemente il token [CLS] dell'intera sequenza Cross-Encoder
        cls_output = outputs.last_hidden_state[:, 0, :] 
        cls_output = self.dropout(cls_output)

        # Passiamo il vettore nelle teste lineari
        logits = [head(cls_output) for head in self.heads]
        
        return torch.stack(logits, dim=1)

### 3. Inizializzazione e DataLoaders
L'ultimo blocco carica fisicamente i file salvati dalla nostra "Fabbrica dei Dati", istanzia i `Dataset`, e crea i `DataLoader` (con batch size = 16) per "nutrire" la GPU in modo efficiente durante l'addestramento. Infine, sposta il modello sulla scheda video (CUDA) pronto per il training.

In [7]:
# 1. Carichiamo la lista delle categorie salvata prima
with open("data_coppie/restaurant_categories.pkl", "rb") as f:
    category_list = pickle.load(f)
num_categories = len(category_list) # 121

# 2. Carichiamo i DataFrame di Train e Dev
df_train = pd.read_pickle("data_coppie/train_restaurant_pairs.pkl")
df_dev = pd.read_pickle("data_coppie/dev_restaurant_pairs.pkl")
df_test = pd.read_pickle("data_coppie/test_restaurant_pairs.pkl")

# 3. Inizializziamo il Tokenizer e i Dataset
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
train_dataset = ACOSPairDataset(df_train, tokenizer)
dev_dataset = ACOSPairDataset(df_dev, tokenizer)
test_dataset = ACOSPairDataset(df_test, tokenizer) 

# 4. Creiamo i DataLoader (Batch size 16 è un buon compromesso tra velocità e VRAM)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dev_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16) 
# 5. Inizializziamo il Modello usando i pesi dello Step 1!
model = ModernBertACOSClassifier("./best_model_restaurant", num_categories)
model.to(device)

print(f"Modello inizializzato! Categorie: {num_categories} | Device: {device}")

Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: ./best_model_restaurant
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
classifier.weight | UNEXPECTED |  | 
classifier.bias   | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Modello inizializzato! Categorie: 13 | Device: mps


## WANDB per lo step 2

In [8]:
# Chiudiamo per sicurezza qualsiasi run precedente rimasta aperta nello stesso notebook
if wandb.run is not None:
    wandb.finish()

WANDB_ENTITY = "cristinatextmining"

# 1. Definizione degli Hyperparameters per lo STEP 2
config_step2 = {
    "learning_rate": 2e-5, # Solitamente per lo Step 2 un LR leggermente più basso è meglio (es. 2e-5)
    "epochs": 40,
    "batch_size": 16,
    "accumulation_steps": 4, # Aggiunto per il tuo training loop ottimizzato!
    "model_name": "answerdotai/ModernBERT-base",
    "dataset": "Restaurant-ACOS", 
    "seed": 42,
    "patience": 5  # Per Early Stopping
}

# 2. Inizializzazione del Run per lo Step 2
wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    config=config_step2,
    # Aggiungiamo "Step2_Class" al nome per distinguerlo dallo Step 1
    name=f"Step2_Class_{config_step2['model_name']}_{config_step2['dataset']}" 
)

print(f" W&B inizializzato per il progetto: {wandb.run.project}")
print(f"Nome della Run attuale: {wandb.run.name}")

wandb: Currently logged in as: cristinatomaciello2001 (cristinatextmining) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


 W&B inizializzato per il progetto: BigData-TextMining-ACOS
Nome della Run attuale: Step2_Class_answerdotai/ModernBERT-base_Restaurant-ACOS


## Train su Sentiment e Categoria

In [10]:
# --- 1. CONFIGURAZIONE AVANZATA MEMORIA ---
# Gradient Checkpointing: abilitato SOLO sul corpo di ModernBERT
model.modernbert.gradient_checkpointing_enable()

# Parametri per simulare un batch size maggiore
accumulation_steps = config_step2.get('accumulation_steps', 4) 
patience = config_step2.get('patience', 5)
patience_counter = 0

# Ottimizzatore AdamW a 8-bit
optimizer = bnb.optim.AdamW8bit(
    model.parameters(), 
    lr=config_step2['learning_rate']
)

# Scaler per Mixed Precision (fondamentale per evitare l'OOM)
scaler = GradScaler() 

# Un bilanciamento molto più equo! Il modello non avrà più il terrore di usare "Invalido"
weights = torch.tensor([2.0, 2.0, 2.0, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

# Calcolo degli step totali per lo scheduler
total_steps = (len(train_loader) // accumulation_steps) * config_step2['epochs']
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps
)

# --- 2. FUNZIONI DI SUPPORTO OTTIMIZZATE (STEP 2) ---

def evaluate_model_step2(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            with autocast(device_type='cuda'):
                # RIMOSSI aspect_spans e opinion_spans
                logits = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = criterion(logits.view(-1, 4), labels.view(-1))
            
            total_loss += loss.item()
            
    return total_loss / len(data_loader)

def train_epoch_step2(model, data_loader, optimizer, scheduler, criterion, device, epoch_idx, scaler, accumulation_steps):
    model.train()
    total_loss = 0
    optimizer.zero_grad() 
    
    loop = tqdm(data_loader, leave=True)
    
    for i, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        with autocast(device_type='cuda'):
            # RIMOSSI aspect_spans e opinion_spans
            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            
            loss = criterion(logits.view(-1, 4), labels.view(-1))
            loss = loss / accumulation_steps 
        
        scaler.scale(loss).backward()
        
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(data_loader):
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        
        real_loss = loss.item() * accumulation_steps
        total_loss += real_loss
        
        wandb.log({"batch_loss": real_loss})
        loop.set_description(f"Epoca {epoch_idx + 1}")
        loop.set_postfix(loss=real_loss)

    return total_loss / len(data_loader)

# --- 3. CICLO DI ADDESTRAMENTO ---

print(f"Training STEP 2 su RESTAURANT: {config_step2['epochs']} epoche | Device: {device}")
print(f"Accumulo Gradienti ogni {accumulation_steps} step | FP16 Attivato")

best_valid_loss_restaurant = float('inf')

for epoch in range(config_step2['epochs']):
    print(f"\n--- Epoca {epoch+1}/{config_step2['epochs']} ---")
    
    # 1. Training
    train_loss_restaurant = train_epoch_step2(
        model, train_loader, optimizer, scheduler, criterion, 
        device, epoch, scaler, accumulation_steps
    )
    
    # 2. Validazione
    valid_loss_restaurant = evaluate_model_step2(model, val_loader, criterion, device)
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print(f"Train Loss: {train_loss_restaurant:.4f} | Valid Loss: {valid_loss_restaurant:.4f}")
    
    # 3. Log metriche epoca su W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss_epoch": train_loss_restaurant,
        "valid_loss_epoch": valid_loss_restaurant
    })
    
    # --- LOGICA EARLY STOPPING & CHECKPOINT ---
    
    if valid_loss_restaurant < best_valid_loss_restaurant:
        best_valid_loss_restaurant = valid_loss_restaurant
        patience_counter = 0  
        
        print(f"Miglior modello trovato (Loss: {best_valid_loss_restaurant:.4f})! Salvataggio...")
        
        # Salvataggio custom model nella cartella dei RESTAURANT
        save_dir = "./best_classifier_restaurant"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            
        torch.save(model.state_dict(), os.path.join(save_dir, "pytorch_model.bin"))
        
    else:
        patience_counter += 1  
        print(f"Nessun miglioramento. Patience: {patience_counter}/{patience}")
        
        if patience_counter >= patience:
            print(f"\nEARLY STOPPING ATTIVATO! Interruzione all'epoca {epoch+1}.")
            break 

print("\nFine Addestramento Step 2 (Restaurant).")
wandb.finish()

Training STEP 2 su RESTAURANT: 40 epoche | Device: cuda
Accumulo Gradienti ogni 4 step | FP16 Attivato

--- Epoca 1/40 ---


Epoca 1: 100%|██████████| 398/398 [01:21<00:00,  4.88it/s, loss=0.184] 


Train Loss: 0.2675 | Valid Loss: 0.2097
Miglior modello trovato (Loss: 0.2097)! Salvataggio...

--- Epoca 2/40 ---


Epoca 2: 100%|██████████| 398/398 [01:21<00:00,  4.90it/s, loss=0.119] 


Train Loss: 0.1697 | Valid Loss: 0.1702
Miglior modello trovato (Loss: 0.1702)! Salvataggio...

--- Epoca 3/40 ---


...

Epoca 11: 100%|██████████| 398/398 [01:22<00:00,  4.80it/s, loss=0.0149] 


Train Loss: 0.0129 | Valid Loss: 0.1280
Nessun miglioramento. Patience: 4/5

--- Epoca 12/40 ---


Epoca 12: 100%|██████████| 398/398 [01:22<00:00,  4.83it/s, loss=0.00632]


Train Loss: 0.0089 | Valid Loss: 0.1340
Nessun miglioramento. Patience: 5/5

EARLY STOPPING ATTIVATO! Interruzione all'epoca 12.

Fine Addestramento Step 2 (Restaurant).


batch_loss,██▇▅▆▇▅▅▇▃▄▅▃▂▃▃▅▃▄▄▂▄▂▂▂▁▂▁▂▂▂▁▁▁▁▁▁▁▁▁
epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_loss_epoch,█▅▄▄▃▂▂▁▁▁▁▁
valid_loss_epoch,█▅▃▂▁▁▁▂▂▂▂▂
batch_loss,0.00632
epoch,12
train_loss_epoch,0.0089
valid_loss_epoch,0.13398


### Test su Sentiment e Categoria
In questa cella effettuiamo il collaudo dello Step 2 sul dataset di Test e andiamo alla ricerca della calibrazione perfetta per le sue predizioni.

Il codice si divide in tre fasi fondamentali:

1. Caricamento e Inferenza (Estrazione Probabilità):
Inizializziamo il modello ModernBertACOSClassifier e carichiamo i pesi addestrati. Successivamente, facciamo passare tutto il set di test all'interno della rete, ma invece di fargli prendere subito una decisione netta, estraiamo le probabilità grezze (Softmax) per ogni classe (Positivo, Negativo, Neutro, Invalido).

2. Perché facciamo un ciclo for (Grid Search)?
Il nostro modello deve decidere se una coppia Aspetto/Opinione è valida e, in caso affermativo, assegnarle un sentimento. Non possiamo fidarci ciecamente della probabilità più alta in assoluto (argmax). A volte i modelli neurali sono troppo "spavaldi" o troppo "timidi".
Attraverso il ciclo for, eseguiamo una Grid Search (Ricerca a Griglia): testiamo pazientemente ogni singola soglia di confidenza dal 50% al 99% (a scatti dell'1%). Per ogni soglia, diciamo al modello: "Accetta questo sentimento SOLO se sei sicuro almeno al X%, altrimenti scarta la coppia nella Classe 3 (Invalido)".

3. In base a cosa scegliamo la soglia migliore?
Come si vede nel codice, salviamo la soglia che massimizza il Macro F1-Score (if current_macro_f1 > best_macro_f1).
Perché proprio il Macro F1 e non il Micro F1 o l'accuratezza globale?
Nei dataset di recensioni, le classi sono spesso molto sbilanciate (es. ci sono tantissime recensioni "Positive" e pochissime "Neutre"). Se usassimo il Micro F1, il modello potrebbe barare, imparando a identificare benissimo solo la classe maggioritaria e ignorando le altre. Il Macro F1, invece, calcola la precisione e il recall per ogni singolo sentimento (Positivo, Negativo, Neutro) in modo indipendente e ne fa la media matematica.

Scegliere la soglia basandoci sul Macro F1 ci garantisce di trovare il punto di equilibrio perfetto: un modello robusto, equilibrato, e capace di riconoscere con la stessa efficacia sia i complimenti che le critiche!


In [9]:
# Chiudiamo run appese
if wandb.run is not None:
    wandb.finish()

WANDB_ENTITY = "cristinatextmining"

wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    name=f"TEST_Step2_{config_step2['model_name']}_{config_step2['dataset']}",
    job_type="test"
)

# Ricreiamo l'architettura del modello
num_categories = len(category_list)
model_test = ModernBertACOSClassifier("./best_model_restaurant", num_categories)

# Carichiamo i pesi dello Step 2
model_path = "./best_classifier_restaurant/pytorch_model.bin"
state_dict = torch.load(model_path, map_location=device)

# USIAMO STRICT=FALSE
missing_keys, unexpected_keys = model_test.load_state_dict(state_dict, strict=False)

print("\n--- CHECK CARICAMENTO PESI ---")
print(f"Chiavi Inaspettate (OK se sono dello Step 1): {len(unexpected_keys)}")
print(f"Chiavi Mancanti (PROBLEMA se sono 'heads'): {len(missing_keys)}")

heads_missing = [k for k in missing_keys if "heads" in k]
if heads_missing:
    print(f"ERRORE CRITICO: Le teste di classificazione non sono state caricate! {heads_missing[:5]}")
else:
    print("SUCCESS: Le teste di classificazione (Sentiment) sono state caricate correttamente.")
    
model_test.to(device)
model_test.eval()

# --- 1. ESTRAZIONE PROBABILITA' ---
print("Estrazione di tutte le probabilità dal modello in corso (attendere)...")

all_probs_list = []
all_true_list = []

with torch.no_grad():
    for batch in test_loader: 
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device) 
        
        # Gestione sicura dell'autocast basata sull'hardware
        if device.type == 'cuda':
            with torch.amp.autocast(device_type='cuda'):
                # RIMOSSI aspect_spans e opinion_spans
                logits = model_test(input_ids, attention_mask)
        else:
            # Per Mac MPS o CPU facciamo il forward pass standard
            logits = model_test(input_ids, attention_mask)
            
        probs = torch.softmax(logits, dim=-1).cpu() 
        
        all_probs_list.append(probs)
        all_true_list.append(labels.cpu())

all_probs = torch.cat(all_probs_list, dim=0) 
all_true = torch.cat(all_true_list, dim=0).numpy().flatten()


# --- 2. GRID SEARCH SUL THRESHOLD (OTTIMIZZATA PER MACRO F1) ---
print("\n🔍 Avvio Grid Search per la migliore Soglia di Confidenza (Macro F1)...")

thresholds_to_test = np.arange(0.50, 1.00, 0.01) 
best_micro_f1 = 0.0
best_macro_f1 = 0.0
best_threshold = 0.0
best_report = ""

target_names = ['Negative (0)', 'Neutral (1)', 'Positive (2)']
labels_to_eval = [0, 1, 2]
# --- AGGIUNGI QUESTE DUE RIGHE QUI ---
prob_invalid = all_probs[:, :, 3] 
best_sentiment_preds = torch.argmax(all_probs[:, :, :3], dim=-1)

# INIZIO CICLO SILENZIOSO
for thresh in thresholds_to_test:
    # 1. Partiamo fiduciosi: diamo a tutto il miglior sentimento predetto
    final_preds = best_sentiment_preds.clone()
    
    # 2. Il Buttafuori: se l'Invalido (Classe 3) supera la soglia, scartiamo la quadrupla
    mask_invalid = prob_invalid > thresh
    final_preds[mask_invalid] = 3
    
    preds_flat = final_preds.numpy().flatten()
    
    current_micro_f1 = f1_score(all_true, preds_flat, labels=labels_to_eval, average='micro')
    current_macro_f1 = f1_score(all_true, preds_flat, labels=labels_to_eval, average='macro')
    
    # ORA VINCE CHI ALZA IL MACRO F1!
    if current_macro_f1 > best_macro_f1:
        best_macro_f1 = current_macro_f1
        best_micro_f1 = current_micro_f1
        best_threshold = thresh
        best_report = classification_report(
            all_true, preds_flat, labels=labels_to_eval, 
            target_names=target_names, zero_division=0
        )

# --- 3. STAMPA DEI RISULTATI VINCITORI E LOG W&B ---
print(f"IL MILGIOR THRESHOLD E'= {best_threshold:.2f} ({(best_threshold*100):.0f}%)")

print("\n--- MIGLIOR CLASSIFICATION REPORT ---")
print(best_report)
print(f"BEST MICRO F1-Score: {best_micro_f1:.4f}")
print(f"CORRISPONDENTE MACRO F1: {best_macro_f1:.4f}")
print("="*50)

wandb.log({
    "best_test_micro_f1": best_micro_f1,
    "best_test_macro_f1": best_macro_f1,
    "optimal_threshold": best_threshold
})
wandb.finish()

Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: ./best_model_restaurant
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
classifier.weight | UNEXPECTED |  | 
classifier.bias   | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



--- CHECK CARICAMENTO PESI ---
Chiavi Inaspettate (OK se sono dello Step 1): 0
Chiavi Mancanti (PROBLEMA se sono 'heads'): 0
SUCCESS: Le teste di classificazione (Sentiment) sono state caricate correttamente.
Estrazione di tutte le probabilità dal modello in corso (attendere)...

🔍 Avvio Grid Search per la migliore Soglia di Confidenza (Macro F1)...
IL MILGIOR THRESHOLD E'= 0.57 (57%)

--- MIGLIOR CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

Negative (0)       0.51      0.53      0.52       205
 Neutral (1)       0.50      0.02      0.04        44
Positive (2)       0.68      0.64      0.66       667

   micro avg       0.64      0.58      0.61       916
   macro avg       0.56      0.40      0.41       916
weighted avg       0.63      0.58      0.60       916

BEST MICRO F1-Score: 0.6074
CORRISPONDENTE MACRO F1: 0.4057


best_test_macro_f1,▁
best_test_micro_f1,▁
optimal_threshold,▁
best_test_macro_f1,0.40569
best_test_micro_f1,0.60741
optimal_threshold,0.57


### Test sulla quadrupla completa (Aspetto-Opinione-Categoria-Sentiment)
In questa cella mettiamo finalmente alla prova l'intera architettura che abbiamo costruito, unendo le forze dello Step 1 (L'Investigatore) e dello Step 2 (Lo Psicologo) per estrarre le quadruple complete direttamente dal testo grezzo.

Questo è il banco di prova definitivo. Il codice esegue i seguenti passaggi fondamentali:

1. Ripristino dei Pesi: Carichiamo le configurazioni e i pesi ottimali (pytorch_model.bin) che i nostri due modelli ModernBERT hanno appreso durante le rispettive fasi di training.

2. L'Estrazione (Step 1): La funzione predict_quadruples_e2e passa la frase al primo modello. Il Layer CRF decodifica direttamente la sequenza perfetta dei tag (senza più bisogno di argmax), isolando le parole scritte. Le due teste binarie valutano invece l'eventuale presenza di elementi impliciti (-1, -1).

3. Il Cross-Encoder in Azione (Step 2): Il codice crea tutte le combinazioni possibili (Prodotto Cartesiano) tra Aspetti e Opinioni trovate. Per ogni coppia, re-tokenizza la frase originale aggiungendo il nostro prompt ("aspect: X opinion: Y") e la passa allo Psicologo.

4. Filtraggio Intelligente: Il modello analizza le 13 categorie simultaneamente. Grazie all'Hard Negative Sampling visto in addestramento, ci fidiamo ciecamente del suo giudizio: rimuoviamo ogni soglia manuale di probabilità e scartiamo solo le coppie che il modello classifica esplicitamente come "Invalide" (Classe 3).

5. Valutazione Exact Match: Confrontiamo le quadruple predette con la Ground Truth (la verità di base) del dataset di test. La valutazione è spietata: per essere considerata corretta (True Positive), la quadrupla predetta deve corrispondere esattamente all'originale in tutte e 4 le dimensioni (Aspetto, Opinione, Categoria, Sentimento).

In [10]:
# ==========================================
# 1. CARICAMENTO DEI MODELLI E PREPARATIVI
# ==========================================
print("Caricamento configurazioni...")
with open("data_coppie/restaurant_categories.pkl", "rb") as f:
    category_list = pickle.load(f)
num_categories = len(category_list)
id2label = {0: 'O', 1: 'B-ASP', 2: 'I-ASP', 3: 'B-OPI', 4: 'I-OPI'}

print("Caricamento Step 1 (L'Investigatore Multi-Task)...")
model_step1 = ModernBertACOS_Extractor(num_labels=5).to(device)
model_step1.load_state_dict(torch.load("./best_multitask_extractor_restaurant/pytorch_model.bin", map_location=device, weights_only=True))
model_step1.eval()

print("Caricamento Step 2 (Lo Psicologo Classificatore)...")
model_step2 = ModernBertACOSClassifier("./best_model_restaurant", num_categories).to(device)
model_step2.load_state_dict(torch.load("./best_classifier_restaurant/pytorch_model.bin", map_location=device, weights_only=True))
model_step2.eval()

# ==========================================
# 2. FUNZIONI DI SUPPORTO PER L'ESTRAZIONE
# ==========================================
def get_spans(tags, b_tag, i_tag):
    spans = []
    start = -1
    for i, tag in enumerate(tags):
        if tag == b_tag:
            if start != -1: spans.append((start, i))
            start = i
        elif tag == i_tag and start != -1: continue
        else:
            if start != -1:
                spans.append((start, i))
                start = -1
    if start != -1: spans.append((start, len(tags)))
    return spans

def predict_quadruples_e2e(text, model_1, model_2, tokenizer, cat_list):
    """La Pipeline End-to-End con filtri severi antidisturbo."""
    words = text.split()
    inputs = tokenizer(words, is_split_into_words=True, return_tensors="pt", truncation=True, max_length=128, padding='max_length').to(device)
    
    # --- FASE 1: L'Investigatore ---
    with torch.no_grad():
        with torch.amp.autocast(device_type=device.type):
            out1 = model_1(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        
    token_preds = out1['token_logits'][0].cpu().numpy()
    
    # usiamo l'argmax classico (sicurezza > 50%) per evitare allucinazioni!
    imp_asp = torch.argmax(out1['imp_asp_logits'], dim=-1)[0].item()
    imp_opi = torch.argmax(out1['imp_opi_logits'], dim=-1)[0].item()
    
    word_ids = inputs.word_ids()
    word_tags = ["O"] * len(words)
    
    for idx, w_id in enumerate(word_ids):
        if w_id is not None and w_id < len(words) and word_tags[w_id] == "O":
            word_tags[w_id] = id2label[token_preds[idx]]
                
    asp_spans = get_spans(word_tags, "B-ASP", "I-ASP")
    opi_spans = get_spans(word_tags, "B-OPI", "I-OPI")
    
    if imp_asp == 1 or len(asp_spans) == 0: asp_spans.append((-1, -1))
    if imp_opi == 1 or len(opi_spans) == 0: opi_spans.append((-1, -1))
    
    asp_spans = list(set(asp_spans))
    opi_spans = list(set(opi_spans))
    
    quadruples = []
    
# --- FASE 2: Lo Psicologo (Versione Cross-Encoder per Ristoranti!) ---
    for a in asp_spans:
        for o in opi_spans:
            
            # Estraiamo le stringhe
            asp_str = " ".join(words[a[0]:a[1]]) if a != (-1, -1) else "null"
            opi_str = " ".join(words[o[0]:o[1]]) if o != (-1, -1) else "null"
            cross_text = f"aspect: {asp_str} opinion: {opi_str}"
            
            # Re-Tokenizziamo al volo per il Cross-Encoder
            pair_inputs = tokenizer(
                text, cross_text, 
                return_tensors="pt", truncation=True, 
                max_length=128, padding='max_length'
            ).to(device)
            
            with torch.no_grad():
                with torch.amp.autocast(device_type=device.type):
                    # Niente più tensori t_a e t_o! Passiamo i nuovi input formattati
                    out2 = model_2(
                        input_ids=pair_inputs['input_ids'], 
                        attention_mask=pair_inputs['attention_mask']
                    )
            
            logits = out2['logits'] if isinstance(out2, dict) else out2 
            probs = torch.softmax(logits[0], dim=-1) 
            
            for cat_idx, prob_dist in enumerate(probs):
                
                prob_invalido = prob_dist[3].item()
                
                # Se la probabilità di Invalido supera il 57%, scartiamo!
                if prob_invalido > 0.57:
                    continue
                
                # Altrimenti, ha passato il controllo! 
                best_sentiment = torch.argmax(prob_dist[:3]).item()
                
                quadruples.append({
                    'aspect_span': a,
                    'opinion_span': o,
                    'category': cat_list[cat_idx],
                    'sentiment': best_sentiment
                })
                    
    return quadruples

# ==========================================
# 3. TEST SULL'INTERO DATASET RESTAURANT
# ==========================================
print("\nAvvio Valutazione End-to-End sul Test Set...")
true_quads = []
pred_quads = []

percorso_file = os.path.join("data_parsing", "test_rest_parsed.pkl")
test_rest_parsed = pd.read_pickle(percorso_file)

for idx, row in tqdm(test_rest_parsed.iterrows(), total=len(test_rest_parsed), desc="Analisi Frasi"):
    text = row['review_text']
    preds = predict_quadruples_e2e(text, model_step1, model_step2, tokenizer, category_list)
    
    p_set = set()
    for q in preds:
        p_set.add((tuple(q['aspect_span']), q['category'], tuple(q['opinion_span']), q['sentiment']))
    pred_quads.append(p_set)
    
    t_set = set()
    for q in row['parsed_quadruples']:
        a = tuple(q.get('span_A', [-1, -1]))
        o = tuple(q.get('span_B', [-1, -1]))
        c = q['category_aspect']
        s = int(q['sentiment'])
        t_set.add((a, c, o, s))
    true_quads.append(t_set)

# Metriche finali
total_pred = sum(len(p) for p in pred_quads)
total_true = sum(len(t) for t in true_quads)
correct = sum(len(p_set.intersection(t_set)) for p_set, t_set in zip(pred_quads, true_quads))

precision = correct / total_pred if total_pred > 0 else 0
recall = correct / total_true if total_true > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("\n" + "="*50)
print("RISULTATI FINALI EXACT MATCH ACOS (RESTAURANT)")
print("="*50)
print(f"Quadruple Reali totali:    {total_true}")
print(f"Quadruple Predette totali: {total_pred}")
print(f"Quadruple Esatte:          {correct}")
print("-" * 50)
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print("="*50)


Caricamento configurazioni...
Caricamento Step 1 (L'Investigatore Multi-Task)...


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
decoder.bias      | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
head.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Caricamento Step 2 (Lo Psicologo Classificatore)...


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: ./best_model_restaurant
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
classifier.weight | UNEXPECTED |  | 
classifier.bias   | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Avvio Valutazione End-to-End sul Test Set...


Analisi Frasi: 100%|██████████| 583/583 [03:01<00:00,  3.21it/s]


RISULTATI FINALI EXACT MATCH ACOS (RESTAURANT)
Quadruple Reali totali:    916
Quadruple Predette totali: 832
Quadruple Esatte:          357
--------------------------------------------------
Precision: 0.4291
Recall:    0.3897
F1-Score:  0.4085
